In [4]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_community.tools import ShellTool
from langgraph.prebuilt import ToolNode
from typing import TypedDict, List
import os
from dotenv import load_dotenv
from github import Github
load_dotenv()


True

In [5]:
# Set up Groq LLM
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  

llm = ChatGroq(
    model="openai/gpt-oss-20b",
)

In [6]:
# Define the state
class CodeReviewState(TypedDict):
    repo_name: str
    pr_number: int
    code_changes: List[str]
    feedback: List[str]
    issues: List[str]
    suggestions: List[str]

In [7]:
# Fetch PR changes
def fetch_pr_changes(state: CodeReviewState):
    repo_name = state["repo_name"]
    pr_number = state["pr_number"]
    
    g = Github(os.getenv("GITHUB_TOKEN"))
    repo = g.get_repo(repo_name)
    pr = repo.get_pull(pr_number)
    
    files = pr.get_files()
    code_changes = []
    for file in files:
        if file.patch:
            code_changes.append(f"File: {file.filename}\n{file.patch}")
    
    return {"code_changes": code_changes}

# Define nodes
def analyze_code(state: CodeReviewState):
    code_changes = state["code_changes"]
    all_code = "\n\n".join(code_changes)
    
    # Read the prompt from prompt.txt
    with open("../prompt.txt", "r", encoding="utf-8") as f:
        prompt_template = f.read()
    
    # Replace the placeholder with the actual code
    prompt = prompt_template.replace("{all_code}", all_code)
    
    response = llm.invoke([SystemMessage(content="You are a code review expert."), HumanMessage(content=prompt)])
    analysis = response.content
    
    # Return the full structured analysis as feedback
    return {"feedback": [analysis]}

def provide_feedback(state: CodeReviewState):
    # The feedback is already generated in analyze_code
    # Format it professionally for GitHub
    feedback = state["feedback"][0] if state["feedback"] else "No feedback generated."
    
    # Add a professional header
    professional_feedback = f"""## 🤖 AI Code Review Analysis

{feedback}

---
*This review was generated automatically by the SmartReview AI agent. Please review the suggestions and address any critical issues before merging.*"""
    
    return {"feedback": [professional_feedback]}

def post_feedback(state: CodeReviewState):
    repo_name = state["repo_name"]
    pr_number = state["pr_number"]
    feedback = state["feedback"][0] if state["feedback"] else "No feedback generated."
    
    g = Github(os.getenv("GITHUB_TOKEN"))
    repo = g.get_repo(repo_name)
    pr = repo.get_pull(pr_number)
    
    pr.create_issue_comment(feedback)
    return {}

In [8]:
# Build the graph
graph = StateGraph(CodeReviewState)

graph.add_node("fetch_pr", fetch_pr_changes)
graph.add_node("analyze", analyze_code)
graph.add_node("feedback", provide_feedback)
graph.add_node("post", post_feedback)

graph.add_edge(START, "fetch_pr")
graph.add_edge("fetch_pr", "analyze")
graph.add_edge("analyze", "feedback")
graph.add_edge("feedback", "post")
graph.add_edge("post", END)

code_review_agent = graph.compile()


In [9]:
# Test the agent with PR
# Example: repo_name = "owner/repo", pr_number = 1
result = code_review_agent.invoke({
    "repo_name": "shrivarshapoojari/SmartReview",  # Replace with actual repo
    "pr_number": 1,  # Replace with actual PR number
    "code_changes": [],
    "feedback": [],
    "issues": [],
    "suggestions": []
})

# Display the result in a readable way
print("=== Code Review Agent Result ===")
print(f"Repository: {result.get('repo_name', 'N/A')}")
print(f"PR Number: {result.get('pr_number', 'N/A')}")
print("\n--- Code Changes ---")
for i, change in enumerate(result.get('code_changes', []), 1):
    print(f"{i}. {change[:100]}...")  # 
print("\n--- Issues ---")
for issue in result.get('issues', []):
    print(f"- {issue}")
print("\n--- Suggestions ---")
for suggestion in result.get('suggestions', []):
    print(f"- {suggestion}")
print("\n--- Feedback ---")
for feedback in result.get('feedback', []):
    print(feedback)

C:\Users\shree\AppData\Local\Temp\ipykernel_33488\428173092.py:6: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(os.getenv("GITHUB_TOKEN"))
C:\Users\shree\AppData\Local\Temp\ipykernel_33488\428173092.py:56: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(os.getenv("GITHUB_TOKEN"))
C:\Users\shree\AppData\Local\Temp\ipykernel_33488\428173092.py:56: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(os.getenv("GITHUB_TOKEN"))


=== Code Review Agent Result ===
Repository: shrivarshapoojari/SmartReview
PR Number: 1

--- Code Changes ---
1. File: research/research.ipynb
@@ -2,7 +2,7 @@
  "cells": [
   {
    "cell_type": "code",
-   "execut...
2. File: test.py
@@ -0,0 +1,53 @@
+from flask import Flask, request, render_template_string
+import sql...

--- Issues ---

--- Suggestions ---

--- Feedback ---
## 🤖 AI Code Review Analysis

**1. Executive summary**  
Top 5 issues (ranked by severity and effort to fix):  

| Rank | Issue | One‑liner remediation |
|------|-------|-----------------------|
| 1 | **SQL Injection** in `/login` | Use parameterised queries (e.g., `cursor.execute("SELECT … WHERE …", (username, password))`). |
| 2 | **Cross‑Site Scripting** in `/search` | Escape or template‑render user input instead of interpolating it into `render_template_string`. |
| 3 | **Missing CSRF protection** on the POST login form | Add Flask‑WTF CSRF token or `@app.before_request` guard. |
| 4 | **Debug mode enabled** (

In [11]:
# Display the graph
 